In [2]:
# !pip install vaderSentiment
# !pip install geopy

In [1]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

from  geopy.geocoders import Nominatim

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

import re
import csv

In [2]:
API_key='4JbRSuWcb4Z7RatSjhYOGgWxX'
API_secret_key='2WIKcZZi5ZMPxyhhQwQqVktikqJPUEWbOfwomI2zAFtE9GDDmv'
access_token='1169140087533309953-hA3EYKtBWP9jShHArjxTImOBsRVbUH'
access_token_secret='4LvSEIZBPcmhEclWL6uKKyyqJOREdwvSd6mVy2vJxh7WB'

In [3]:
geolocator = Nominatim()
sentiment = SentimentIntensityAnalyzer() 

'''
#run only the first time
column_headers=['User_Id','Created_at','Location','Latitude','Longitude','Tweet_text','Neutral','Positive','Negative','Sentiment_compound']

with open('tweets.csv','w') as file:
    writer=csv.DictWriter(file,fieldnames=column_headers)
    writer.writeheader()'''
    

H:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


"with open('tweets.csv','w') as file:\n    writer=csv.DictWriter(file,fieldnames=column_headers)\n    writer.writeheader()"

In [4]:
def deEmojify(text):
    #Removing non-ASCII characters (emoji characters)
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [5]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, tweet):
        #Extracting tweets
        
        if tweet.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        user_location = deEmojify(tweet.user.location)
        if 'India' in str(user_location):
            try:
                id_str = tweet.id_str
                created_at = tweet.created_at
                
                #Pre-processing the text
                text = deEmojify(tweet.text)    
                #Removing RT, links and special characters in the tweet text
                text=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.text).split())
                text=' '.join(re.sub("RT"," ",text).split())
                
                #Getting the co-ordinates of the user
                loc = geolocator.geocode(user_location)
                longitude = loc.longitude
                latitude = loc.latitude
                
                #Sentiment analysis by vaderSentiment
                sentiment_tweet = sentiment.polarity_scores(text)
                tweet_neu=sentiment_tweet['neu']
                tweet_pos=sentiment_tweet['pos']
                tweet_neg=sentiment_tweet['neg']
                tweet_compound=sentiment_tweet['compound']
                
                with open('tweets.csv','a') as file:
                    writer=csv.DictWriter(file,fieldnames=column_headers)
                    new_tweet={
                        'User_Id':id_str,
                        'Created_at':created_at,
                        'Location':user_location,
                        'Latitude':latitude,
                        'Longitude':longitude,
                        'Tweet_text':text,
                        'Neutral':tweet_neu,
                        'Positive':tweet_pos,
                        'Negative':tweet_neg,
                        'Sentiment_compound':tweet_compound
                    }
                    writer.writerow(new_tweet)
                
                print(created_at,"\n",text,"\n",user_location,longitude,latitude)
                print(tweet_neu,tweet_pos,tweet_neg,tweet_compound)
                
            except:
                return True

    def on_error(self, status_code):
        # Since Twitter API has rate limits, this stop srcraping data as it exceed to the thresold.
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [ ]:
auth=tweepy.OAuthHandler(API_key,API_secret_key)
auth.set_access_token(access_token,access_token_secret)
api=API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = ['corona','covid-19','lockdown'])
                #not sure whether this location is doing anything on the tweets
                #,locations=[68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078])

2020-06-22 15:08:38 
 Updates 1 Delhi Police opposes Safoora Zargar s bail stating that 39 deliveries in Delhi prison in last 10 years Zargar 
 Mumbai, India 72.8353355 18.9387711
0.858 0.0 0.142 -0.5106
2020-06-22 15:08:39 
 MJ Only 6 000 active COVID 19 cases in a state which has around 24 crore population UP CM Commendable 
 New Delhi, India 77.2022662 28.6141793
0.876 0.124 0.0 0.4019
2020-06-22 15:08:39 
 MJ Only 6 000 active COVID 19 cases in a state which has around 24 crore population UP CM Commendable 
 Bidar, India 77.333333 18.083333
0.876 0.124 0.0 0.4019
2020-06-22 15:08:39 
 ayurveda Excited to see Waited for it during lockdown Missed it 
 Lucknow, India 80.9346001 26.8381
0.662 0.176 0.162 0.0516
2020-06-22 15:08:40 
 Before conducting exams Please open your parliament All mode of transportation stop area getting sealed after gett 
 Sagardighi, West Bengal, India 88.1332168517106 24.308991749999997
0.769 0.118 0.113 0.0258
2020-06-22 15:08:43 
 I am a student of BBA Sir 

2020-06-22 15:09:43 
 Updates 1 Delhi Police opposes Safoora Zargar s bail stating that 39 deliveries in Delhi prison in last 10 years Zargar 
 Chennai, India 80.2838331 13.0801721
0.858 0.0 0.142 -0.5106
2020-06-22 15:09:44 
 When I step on the weighing machine after lockdown My weighing machine to me 
 Bengaluru, India 77.5912997 12.9791198
1.0 0.0 0.0 0.0
2020-06-22 15:09:44 
 bitch soul 72 bitch0 Face to asha bna lete km se km Flowers v bhut du 
 India 78.6677428 22.3511148
0.798 0.0 0.202 -0.5859
2020-06-22 15:09:45 
 There was a lockdown in the country because of corona Meanwhile the Delhi police were making arrest students On 
 Jaisalmer, India 70.7785056232077 27.02801615
0.882 0.0 0.118 -0.34
2020-06-22 15:09:49 
 Updates 1 Delhi Police opposes Safoora Zargar s bail stating that 39 deliveries in Delhi prison in last 10 years Zargar 
 New Delhi, India 77.2022662 28.6141793
0.858 0.0 0.142 -0.5106
2020-06-22 15:09:50 
 Happy to be back to normal life after almost 25 days We live

2020-06-22 15:10:21 
 karve In democracy shouldn t politicians be held accountable for their decisions Why do politicians have power but no accountabil 
 India 78.6677428 22.3511148
0.872 0.0 0.128 -0.4215
2020-06-22 15:10:22 
 Prabhas RRR isn t a pan India film Then how they released rc look in lockdown fans losing their interest 
 Andhra Pradesh, India 80.1863809 15.9240905
0.763 0.127 0.11 0.1027
2020-06-22 15:10:22 
 News Holding RathYatra in Puri amid COVID 19 pandemic is a challenge Festival must be celebrated by striking perfect balance be 
 India 78.6677428 22.3511148
0.588 0.412 0.0 0.8979
2020-06-22 15:10:24 
 NEWS NASCAR industry sources tell me Richard Petty will attend today s race at Talladega to stand in solidarity w 
 Indianapolis, IN -86.1583502 39.7683331
0.818 0.1 0.082 0.1027
2020-06-22 15:10:24 
 This EORS I couldn t buy blazer because i used my money for poor people who has been stuck here due to lockdown 
 India 78.6677428 22.3511148
0.805 0.0 0.195 -0.6249
2020-

2020-06-22 15:11:01 
 Corona update Total active cases worldwide 37 lacs total recovered 49 lacs Total serious cases as of now are 57 K Less 
 Greater Faridabad, India 77.24271225505706 28.54452615
0.813 0.122 0.065 0.34
2020-06-22 15:11:02 
 Let not India s surender mislead you into thinking that India has won the Corona war The battle against Coronavirus is ver 
 Uttar Pradesh, India 80.859666 27.1303344
0.651 0.127 0.223 -0.4215
2020-06-22 15:11:03 
 UK accused of trying to thwart rival COVID 19 apps 
 India 78.6677428 22.3511148
0.804 0.0 0.196 -0.296
2020-06-22 15:11:05 
 I just can t get over how much weight she has lost in lockdown ShehnaazGill HungamaWithShehnaaz gill 
 Punjab , India 75.5004841 30.9293211
0.874 0.0 0.126 -0.3182
2020-06-22 15:11:06 
 Kerala clamps down on travel of aged and kids as Covid 19 cases rise 
 New Delhi, India 77.2022662 28.6141793
1.0 0.0 0.0 0.0
2020-06-22 15:11:06 
 How hyperlocal OOH content can help in doing brand recall Want to know how busines